Rachel's recommendation:
- Try [Bag of Visual Words](https://towardsdatascience.com/bag-of-visual-words-in-a-nutshell-9ceea97ce0fb) before SIFT

##Set Up

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import shutil #save images
from tqdm import tqdm #progress bar
import cv2
import matplotlib.pyplot as plt
import math
import random


from skimage.exposure import rescale_intensity
from skimage.transform import rescale, rotate
from skimage.color import rgb2gray
from skimage.feature import SIFT, match_descriptors
from skimage.data import camera
from skimage.transform import rotate
from sklearn.metrics import mean_squared_error
from scipy.stats import wasserstein_distance
from scipy.spatial.distance import cosine
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfTransformer


from skimage import color, exposure
from sklearn.decomposition import PCA


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Authenticate.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Install Cloud Storage FUSE.
!echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt -qq update && apt -qq install gcsfuse

deb https://packages.cloud.google.com/apt gcsfuse-jammy main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1022  100  1022    0     0   7364      0 --:--:-- --:--:-- --:--:--  7405
OK
49 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://packages.cloud.google.com/apt/dists/gcsfuse-jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 11.0 MB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unsele

In [ ]:
# Mount a Cloud Storage bucket or location
mount_path = "281-project-d5d834b8-2d7c-11ef-91d5-b89a2a9d8518"
local_path = f"/mnt/gs/{mount_path}"
downsampled_path = f"{local_path}/raw-data/downsampled-data"
preprocessed_path = f"{local_path}/raw-data/preprocessed_data"
preprocessed_red_path = f"{local_path}/raw-data/preprocessed_data_red"
low_res_path = f"{local_path}/raw-data/low_resolution_images"

!mkdir -p {local_path}
!gcsfuse --implicit-dirs {mount_path} {local_path}

{"timestamp":{"seconds":1721945731,"nanos":82779256},"severity":"INFO","message":"Start gcsfuse/2.3.2 (Go version go1.22.4) for app \"\" using mount point: /mnt/gs/281-project-d5d834b8-2d7c-11ef-91d5-b89a2a9d8518\n"}
{"timestamp":{"seconds":1721945731,"nanos":83085372},"severity":"INFO","message":"GCSFuse mount command flags: {\"AppName\":\"\",\"Foreground\":false,\"ConfigFile\":\"\",\"MountOptions\":{},\"DirMode\":493,\"FileMode\":420,\"Uid\":-1,\"Gid\":-1,\"ImplicitDirs\":true,\"OnlyDir\":\"\",\"RenameDirLimit\":0,\"IgnoreInterrupts\":true,\"CustomEndpoint\":null,\"BillingProject\":\"\",\"KeyFile\":\"\",\"TokenUrl\":\"\",\"ReuseTokenFromUrl\":true,\"EgressBandwidthLimitBytesPerSecond\":-1,\"OpRateLimitHz\":-1,\"SequentialReadSizeMb\":200,\"AnonymousAccess\":false,\"MaxRetrySleep\":30000000000,\"StatCacheCapacity\":20460,\"StatCacheTTL\":60000000000,\"TypeCacheTTL\":60000000000,\"KernelListCacheTtlSeconds\":0,\"HttpClientTimeout\":0,\"MaxRetryDuration\":-1000000000,\"RetryMultiplier\"

In [ ]:
#check mounting
os.listdir(f"{local_path}/raw-data/")

['downsampled-data',
 'low_resolution_images',
 'orinoquia-camera-traps',
 'preprocessed_data',
 'preprocessed_data_red',
 'test_data',
 'train_data',
 'validation_data']

##Specify input & create output directory

In [ ]:
#specifiy input image folder
input_path = os.path.join(local_path, 'train_data_preprocessed')

# Create output directory if it doesn't exist
output_path = os.path.join(local_path, 'features')
os.makedirs(output_path, exist_ok=True)

#test makedir
os.listdir(f"{local_path}")

['feature_matrices',
 'features',
 'preprocessed-data',
 'raw-data',
 'train_data_preprocessed']

In [ ]:
input_files = os.listdir(input_path)
print("Number of Images: ", len(input_files))
print("Sample file name: ", input_files[0])

Number of Images:  6254
Sample file name:  A01_02260289.JPG


In [ ]:
clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))
sift = cv2.SIFT_create(nOctaveLayers=5, sigma=5, contrastThreshold=0.01, edgeThreshold=15) #sigma 3, noctave 5, contrastThreshold=0.01
descriptors = []
filenames = []

## Helper functions

In [ ]:
#BOVW
def extract_bovw_features(img, vocabulary, kmeans):
    keypoints, descriptor = sift.detectAndCompute(img, None)
    if descriptor is None or descriptor.size == 0:
        return np.zeros(len(vocabulary))

    # Assign each descriptor to its nearest visual word
    words = kmeans.predict(descriptor)

    # Compute histogram of visual words
    histogram, _ = np.histogram(words, bins=len(vocabulary), range=(0, len(vocabulary)))

    return histogram

## Preprocessing + SIFT

In [ ]:
# Select 10% of the images for sampling
all_files = os.listdir(input_path)
num_files_to_process = max(1, int(len(all_files) * 0.1))
selected_files = random.sample(all_files, num_files_to_process)

In [ ]:
#with selected files
for file in tqdm(selected_files): #change to all_files for training.

    # Read image, convert to grayscale, rescale, and adjust data type
    img = plt.imread(f"{input_path}/{file}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (0,0), fx=0.1, fy=0.1)
    img = img.astype(np.uint8)
    img = clahe.apply(img)

    # Detect SIFT features and compute descriptors
    keypoints, descriptor = sift.detectAndCompute(img, None)
    if descriptor is None or descriptor.size == 0:
        continue
    descriptors.append(descriptor)
    filenames.append(file)

100%|██████████| 625/625 [02:17<00:00,  4.54it/s]


## BOVW with K-means

In [ ]:
# Convert list of descriptors to numpy array
descriptors = np.concatenate(descriptors)

# Define number of clusters for k-means
num_clusters = int(math.sqrt(len(selected_files)))#rule of thumb is sqrt of num images
print(f'Number of clusters: {num_clusters}, and number of input files: {len(selected_files)}')

# Fit k-means model to descriptors
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(descriptors)

# Define visual vocabulary
vocabulary = kmeans.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Number of clusters: 25, and number of input files: 625


## Visualize SIFT and related histogram

In [ ]:
# Randomly select 10 images from the selected files for visualization
visualization_files = random.sample(selected_files, min(10, len(selected_files)))

for file in visualization_files:
  # Find species name for visualization files in order to add a label using train_downsampled.csv file for label
    image_labels = pd.read_csv('/content/drive/MyDrive/w281_Project/0_data/train_downsampled.csv')
    common_name = image_labels[image_labels['filename_new'] == file]['common_name'].values
    common_name = common_name[0]

    # Read image, convert to grayscale, rescale, and adjust data type
    img = plt.imread(f"{input_path}/{file}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (0,0), fx=0.1, fy=0.1)
    img = img.astype(np.uint8)
    img = clahe.apply(img)

    # Detect SIFT features and compute descriptors
    keypoints, descriptor = sift.detectAndCompute(img, None)

    # Extract BoVW features
    bovw_features = extract_bovw_features(img, vocabulary, kmeans)
    if bovw_features is None:
        continue

    # Visualize the image, SIFT features, and BoVW histogram
    plt.figure(figsize=(12, 6))

    # Display the image
    plt.subplot(1, 3, 1)
    plt.imshow(img, cmap='gray')
    plt.title(f'Low-res Image: {common_name}')
    plt.axis('off')

    # Display the SIFT features
    plt.subplot(1, 3, 2)
    img_with_keypoints = cv2.drawKeypoints(img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imshow(img_with_keypoints, cmap='gray')
    plt.title(f'SIFT Features: {common_name}')
    plt.axis('off')

    # Display the BoVW histogram
    plt.subplot(1, 3, 3)
    plt.bar(range(len(bovw_features)), bovw_features)
    plt.title(f'BoVW Histogram: {common_name}')
    plt.xlabel('Visual Words')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# while True:pass


KeyboardInterrupt: 